In [ ]:
# Standard libraries
import sys

# External libraries
import lightkurve as lk
import pandas as pd
import numpy as np
from astropy import units as u
from astropy.convolution import convolve, Box1DKernel
from echelle import interact_echelle

# Internal libraries
sys.path.append("../Scripts/")
from utils import *

In [ ]:
input_data = "../Data/Comparison/dht_comparison_data.csv"
output_file = input_data

In [ ]:
measurement_sheet = pd.read_csv(input_data)
measurement_sheet

In [ ]:
# Constants
cache = "../cache"

# Change this value to select the star to study
star_index = 5

# Initial parameters
target = measurement_sheet["Main ID"][star_index]
# Change to True if want to use fast 20-second cadence data
fast = False
# Prior numax and deltanu
numax = measurement_sheet["numax (muHz)"][star_index]
deltanu = measurement_sheet["dnu (muHz)"][star_index]
min_freq = 0.3*numax
max_freq = 1.7*numax

In [ ]:
%matplotlib inline
# Download data
lc = grab_lightcurve(target, cache, fast)
lc.scatter();

In [ ]:
%matplotlib inline
# Convert to periodogram
pg = lc.to_periodogram(
    method="lombscargle",
    normalization="amplitude",
    minimum_frequency=min_freq,
    maximum_frequency=max_freq,
    oversample_factor=10,
    freq_unit=u.microhertz
)
pg.plot();

# Flattened SNR diagram
snr = pg.flatten()
snr.plot();

In [ ]:
%matplotlib notebook
# Smooth and interact echelle
smooth_width = 2
smooth_frequency = pg.frequency.to_value()
smooth_amplitude = convolve(pg.power, Box1DKernel(smooth_width))
interact_echelle(smooth_frequency, smooth_amplitude, 0.5*deltanu, 1.5*deltanu, step=0.01)

In [ ]:
# Check the star's parameters
measurement_sheet.loc[star_index]

In [ ]:
# Uncomment to manually set new measured values
# measurement_sheet.loc[star_index, "dnu (muHz)"] = 6.18
# measurement_sheet.loc[star_index, "dnu error (muHz)"] = 0.04

In [ ]:
# Save to file
measurement_sheet.to_csv(output_file, index=False)

In [ ]:
# Final check
measurement_sheet